In [ ]:
import cv2
import numpy as np

# Load the image and read the world coordinates
image = cv2.imread('house1.png')
world_coords = np.loadtxt('coords.txt')

# Display the image and allow user to select 10 points
points = []

def select_point(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN and len(points) < 10:
        points.append((x, y))
        cv2.circle(image, (x, y), 5, (0, 0, 255), -1)
        cv2.putText(image, str(len(points)), (x + 10, y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
        cv2.imshow("Image", image)

cv2.namedWindow("Image")
cv2.setMouseCallback("Image", select_point)

while True:
    cv2.imshow("Image", image)
    if cv2.waitKey(1) == 27 or len(points) == 10:  # Press Esc or select 10 points to exit
        break

cv2.destroyAllWindows()

# Convert selected points to numpy array
selected_points = np.array(points)

# Task 1: Find the closest intersection point for each selected point
intersection_points = []

for x, y in selected_points:
    min_distance = float('inf')
    closest_point = None

    for X, Y, _, _ in world_coords:
        distance = np.sqrt((X - x) ** 2 + (Y - y) ** 2)

        if distance < min_distance:
            min_distance = distance
            closest_point = (X, Y)

    intersection_points.append(closest_point)

intersection_points = np.array(intersection_points)

# Task 2: Reconstruct the projection matrix P using the DLT algorithm
A = np.zeros((2 * len(intersection_points), 12))

for i, (X, Y) in enumerate(intersection_points):
    x, y = selected_points[i]
    A[2 * i, :3] = [X, Y, 1]
    A[2 * i, 6:9] = [-x * X, -x * Y, -x]

    A[2 * i + 1, 3:6] = [X, Y, 1]
    A[2 * i + 1, 9:12] = [-y * X, -y * Y, -y]

_, _, V = np.linalg.svd(A)
P = V[-1].reshape(3, 4)

# Task 3: Recover the camera calibration matrix K, the camera orientation R, and the camera center C from P
K, R = np.linalg.qr(P[:, :3])
K /= K[2, 2]  # Ensure that K has a positive diagonal

T = np.linalg.inv(K) @ P[:, 3]
C = -R.T @ T

print("Camera Calibration Matrix K:")
print(K)
print("\nCamera Orientation R:")
print(R)
print("\nCamera Center C:")
print(C)
